<a href="https://colab.research.google.com/github/polash102/Data-Mining_caption-comment-analysis/blob/main/lab1_438_rifatsir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install / import packages ----------------------------------
!pip install opencv-python-headless --quiet

import cv2
import numpy as np
import urllib.request
import os

# 2. Download royalty-free test video ---------------------------
# Changed URL to a publicly accessible one
VIDEO_URL = "https://www.learningcontainer.com/wp-content/uploads/2020/05/sample-mp4-file.mp4"
VIDEO_PATH = "sample-mp4-file.mp4" # Also updated video path to match the new video

if not os.path.exists(VIDEO_PATH):
    print("Downloading video...")
    urllib.request.urlretrieve(VIDEO_URL, VIDEO_PATH)
    print("Download complete.")
else:
    print("Video already exists.")

# 3. Open video streams ------------------------------------------
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:
    fps = 24  # fallback fps

panel_size = (320, 240)
grid_width = panel_size[0] * 3  # 3 columns: Original, HE, CLAHE
grid_height = panel_size[1] * 2 # 2 rows: frames + histograms

out = cv2.VideoWriter(
    "output_hist_eq_clahe_grid.mp4",
    cv2.VideoWriter_fourcc(*'mp4v'),
    fps,
    (grid_width, grid_height)
)

# 4. Helper function: Draw RGB histogram panel -------------------
def make_histogram_panel(bgr_frame, size=(320, 240), bins=64):
    panel = np.zeros((size[1], size[0], 3), dtype=np.uint8)
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]  # B, G, R
    for i, color in enumerate(colors):
        hist = cv2.calcHist([bgr_frame], [i], None, [bins], [0, 256])
        cv2.normalize(hist, hist, 0, size[1], cv2.NORM_MINMAX)
        hist = hist.flatten()
        bin_width = int(size[0] / bins)
        for j in range(bins):
            x1 = j * bin_width
            x2 = (j + 1) * bin_width
            y = size[1] - int(hist[j])
            cv2.rectangle(panel, (x1, y), (x2 - 1, size[1]), color, -1)
    return panel

# 5. RMS contrast metric ------------------------------------------
def rms_contrast(gray_img):
    mean = np.mean(gray_img)
    return np.sqrt(np.mean((gray_img - mean) ** 2))

# 6. Initialize CLAHE ---------------------------------------------
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

# 7. Main processing loop ----------------------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize original frame for grid display
    orig_resized = cv2.resize(frame, panel_size)

    # Convert to YCrCb and split channels
    ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(ycrcb)

    # Global Histogram Equalization
    y_eq = cv2.equalizeHist(y)

    # CLAHE (Adaptive HE)
    y_clahe = clahe.apply(y)

    # Merge channels and convert back to BGR
    frame_eq = cv2.cvtColor(cv2.merge([y_eq, cr, cb]), cv2.COLOR_YCrCb2BGR)
    frame_clahe = cv2.cvtColor(cv2.merge([y_clahe, cr, cb]), cv2.COLOR_YCrCb2BGR)

    # Resize processed frames
    eq_resized = cv2.resize(frame_eq, panel_size)
    clahe_resized = cv2.resize(frame_clahe, panel_size)

    # Compute RMS contrast for luminance channels
    rms_orig = rms_contrast(y)
    rms_eq = rms_contrast(y_eq)
    rms_clahe = rms_contrast(y_clahe)

    # Generate histogram panels
    hist_orig = make_histogram_panel(orig_resized, size=panel_size)
    hist_eq = make_histogram_panel(eq_resized, size=panel_size)
    hist_clahe = make_histogram_panel(clahe_resized, size=panel_size)

    # Compose 3x2 grid:
    # Row 1: Original | Global HE | CLAHE
    # Row 2: Hist Orig | Hist HE  | Hist CLAHE
    top_row = np.hstack([orig_resized, eq_resized, clahe_resized])
    bottom_row = np.hstack([hist_orig, hist_eq, hist_clahe])
    grid = np.vstack([top_row, bottom_row])

    # Overlay RMS contrast text on the top row frames
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(grid, f"Orig RMS: {rms_orig:.2f}", (10, 20), font, 0.6, (255, 255, 255), 1)
    cv2.putText(grid, f"HE RMS: {rms_eq:.2f}", (panel_size[0] + 10, 20), font, 0.6, (255, 255, 255), 1)
    cv2.putText(grid, f"CLAHE RMS: {rms_clahe:.2f}", (2 * panel_size[0] + 10, 20), font, 0.6, (255, 255, 255), 1)

    # Write the grid frame to output video
    out.write(grid)

# 8. Release everything ------------------------------------------
cap.release()
out.release()
print("Processing complete. Output saved as output_hist_eq_clahe_grid.mp4")

Download complete.
Processing complete. Output saved as output_hist_eq_clahe_grid.mp4
